In [3]:
import pickle as pkl
import pandas as pd
with open("SelectedJobData_0801-1001.pkl", "rb") as f:
    object = pkl.load(f)
    
df = pd.DataFrame(object)
df.to_csv(r'data.csv')

In [41]:
import csv
import numpy as np
import pandas as pd
import datetime as dt

# extract fields
fields = ['_id', 'group', 'owner', 'job_name', 'job_number', 'slots', 'mem', 'submission_time', 'start_time', 'end_time']
df = pd.read_csv('data/0_raw_data.csv', skipinitialspace=True, usecols=fields)

# save to
df.to_csv(r'data/1_selection_data.csv', index = False)

df


['_id', 'group', 'owner', 'job_name', 'job_number', 'submission_time', 'start_time', 'end_time', 'mem', 'slots']


,_id,group,owner,job_name,job_number,submission_time,start_time,end_time,mem,slots
0,ObjectId(5f24fe11c14d4f44f577352e),phys,msanati,MPI_Test_Job,1925113,2020-07-30 07:15:06,2020-07-30 07:15:04,2020-08-01 05:29:36,2.921355e+07,108
1,ObjectId(5f24fe11c14d4f44f577352f),phys,msanati,MPI_Test_Job,1925113,2020-07-30 07:15:06,2020-07-30 07:15:04,2020-08-01 05:29:36,2.918893e+07,108
2,ObjectId(5f24fe11c14d4f44f577352a),chem,juandomi,phi_1-2.6,1927135,2020-08-01 02:08:08,2020-08-01 03:37:09,2020-08-01 05:26:28,8.201353e+05,36
3,ObjectId(5f24fe11c14d4f44f577352b),chem,juandomi,phi_1-2.1,1927130,2020-08-01 02:08:02,2020-08-01 03:16:27,2020-08-01 05:26:54,9.722759e+05,36
4,ObjectId(5f24fe11c14d4f44f577352c),ME,hge,Gm15ctc3,1927227,2020-08-01 05:09:45,2020-08-01 05:09:45,2020-08-01 05:29:16,8.610316e+05,72
...,...,...,...,...,...,...,...,...,...,...
457481,ObjectId(5f804788c14d4f0ff4b52a77),bio,jenjense,FSC-UShist9,753735,2020-09-30 01:22:30,2020-10-09 05:00:04,2020-10-09 11:08:36,1.563953e+03,1
457482,ObjectId(5f820988c14d4f0ff4b93ce6),bio,jenjense,FSC-USbott9,753732,2020-09-30 01:10:04,2020-10-10 05:00:36,2020-10-10 18:47:27,3.510346e+03,1
457483,ObjectId(5f819909c14d4f0ff4b83f17),bio,jenjense,FSC-UShist8,753734,2020-09-30 01:19:44,2020-10-10 05:00:09,2020-10-10 11:12:08,1.579265e+03,1
457484,ObjectId(5f83af6ac14d4f0ff4bd2c85),CEislam,rafiisla,ACE2-SGP.2.R18,1960585,2020-09-09 13:31:27,2020-10-11 11:58:26,2020-10-12 00:15:20,7.484000e-01,360


#### Show difference in time

In [99]:
# remove the incorrect records ------------------

df = pd.read_csv("data/1_selection_data.csv", parse_dates=True)

# Tq = log( start_time - submit_time )
df["queuetime"] = (pd.to_datetime(df["start_time"], format='%Y-%m-%d %H:%M:%S') - pd.to_datetime(df["submission_time"], format='%Y-%m-%d %H:%M:%S')).dt.total_seconds()

# Tr = log( end_time - start_time )
df["runtime"] = (pd.to_datetime(df["end_time"], format='%Y-%m-%d %H:%M:%S') - pd.to_datetime(df["start_time"], format='%Y-%m-%d %H:%M:%S')).dt.total_seconds()

# Remove negative records and convert 0 -> 0.[0]1 ------------------

df = df[(df.mem >= 0.0) & (df.slots >= 0.0) & (df.queuetime >= 0.0) & (df.runtime >= 0.0)]

df.to_csv(r'data/2_non_neg.csv', index = False)

print(df.columns.tolist())

['_id', 'group', 'owner', 'job_name', 'job_number', 'submission_time', 'start_time', 'end_time', 'mem', 'slots', 'queuetime', 'runtime']


In [109]:
from pandas import read_csv
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler

attr = ['mem', 'slots', 'queuetime', 'runtime']

df["slots"] = df["slots"] / 36

# retrieve just the numeric input values
data = df[attr].to_numpy()

# perform a robust scaler transform of the dataset
trans = StandardScaler()
data = trans.fit_transform(data)

# convert the array back to a dataframe
dataset = DataFrame(data)

features = ['N_n', 'M_u', 'T_q', 'T_r']

dataset.columns = features

dataset.to_csv(r'data/scaled.csv', index = False)

#### Just data

In [110]:
from pandas import read_csv
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("data/2_non_neg.csv", parse_dates=True)

attr = ['mem', 'slots', 'queuetime', 'runtime']

df["slots"] = df["slots"] / 36

# retrieve just the numeric input values
data = df[attr].to_numpy()

# convert the array back to a dataframe
dataset = DataFrame(data)

features = ['N_n', 'M_u', 'T_q', 'T_r']

dataset.columns = features

dataset.to_csv(r'data/data.csv', index = False)

#### Log

In [106]:
# read data
df = pd.read_csv("data/2_non_neg.csv", parse_dates=True)

df = df.replace([0],0.0000000001)

# Nn = log( nproc/Cn )
df["N_n"] = np.log(df["slots"]/36)

# Mu = log( mem_used )
df["M_u"] = np.log(df["mem"])

# Tq = log( start_time - submit_time )
df["T_q"] = np.log(df["queuetime"])

# Tr = log( end_time - start_time )
df["T_r"] = np.log(df["runtime"])

df.drop(['submission_time', 'start_time', 'end_time', 'mem', 'slots', 'queuetime', 'runtime'], inplace=True, axis=1)

df.to_csv(r'data/log_data.csv', index = False)

print(df.columns.tolist())


['_id', 'group', 'owner', 'job_name', 'job_number', 'N_n', 'M_u', 'T_q', 'T_r']


#### Helper functions

In [34]:
# find the lowest non-zero
df[df['mem'].gt(0)].min()

group                              AFS
owner                         aagudelo
job_name                            A1
job_number                      749322
submission_time    2020-07-23 15:16:18
start_time         2020-07-27 22:14:54
end_time           2020-08-01 00:00:38
mem                             0.0001
slots                                1
N_nodes                       -3.58352
dtype: object

In [82]:
# subtract date time
import datetime as dt

start = job_accounting["start_time"][0]
submit = job_accounting["submission_time"][0]
print(start, submit)

a = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
b = dt.datetime.strptime(submit, '%Y-%m-%d %H:%M:%S')

print(a-b)

2020-07-30 07:15:04 2020-07-30 07:15:06
-1 day, 23:59:58


In [65]:
import pandas as pd
# Smaller data

rows = 5000

df1 = pd.read_csv("data/2_non_neg.csv", skipinitialspace=True)
df1['id'] = df1['group'] + "_" + df1['_id'].str.slice(-5,-1)  
df1a = df1['id'].head(rows)

df2 = pd.read_csv("data/data_all.csv", skipinitialspace=True)
df2 = df2.head(rows)

df3 = pd.concat([df1a, df2], axis=1, ignore_index=True)
df3.columns = ['name', 'N_n', 'M_u', 'T_q', 'T_r']

df3.to_csv(r'data/data.csv', index = False)



In [53]:
import pandas as pd

df = pd.read_csv('data/1_selection_data.csv', skipinitialspace=True)

df['_id'][1][-5:-1]
# pd.unique(df['owner']).size

'352f'

In [71]:
df = pd.read_csv('data/data.csv', skipinitialspace=True)

df[~df.name.str.contains('\w')].name.count()

0